# Assignment Prompt and Generation


This notebook demonstrates how to generate unique assignment prompts and sample student essays using the OpenAI API. The setup includes essential libraries, environment configuration, prompt generation, and essay generation with randomized quality levels.

This notebook generates assignment prompts and sample student essays for specified grade levels and subjects using the OpenAI API.

### Outline:
1. **Setup**
   - Import required libraries.
   - Load environment variables for API access.
   - Define quality levels for essay grading.

2. **Functions**
   - **generate_prompts**: Generates 10 assignment prompts for each grade-subject pair.
   - **generate_sample_essay**: Creates a sample essay for each prompt with a randomly assigned quality level.

3. **Execution**
   - Define grade-subject pairs.
   - Generate and display prompts.
   - Generate and display sample essays with random quality levels.
   
### Requirements:
- **API Key**: Ensure that `OPENAI_API_KEY` is set in the `.env` file in the project directory.
- **Installed Libraries**:
  - `openai`
  - `IPython.display`
  - `python-dotenv`
  - `re` (comes with Python)
  - `random` (comes with Python)








In [1]:
# Import necessary libraries
import openai  # OpenAI API for generating text based on prompts
from IPython.display import display, Markdown  # Markdown display tools for Jupyter Notebooks
import os  # OS library for file and directory interactions
from dotenv import load_dotenv, find_dotenv  # dotenv utilities for managing environment variables
import re  # Regular expressions library for text pattern matching and formatting
import random  # Random module for generating randomized values, such as quality levels

# Function to convert text to Markdown format, allowing formatted display in Jupyter Notebooks
def to_markdown(text):
    # Wraps text in Markdown format for enhanced readability and presentation in Jupyter Notebooks
    return Markdown(text)


In [2]:
# Automatically locate and load the .env file with environment variables
env_path = find_dotenv()  # Searches for the .env file in the project directory
load_dotenv(dotenv_path=env_path)  # Loads the environment variables from .env

# Set the OpenAI API key from environment variable (pulled from the .env file)
openai.api_key = os.getenv('OPENAI_API_KEY')


### Function: `generate_prompts`

#### Description:
- This function generates a list of assignment prompts tailored to specific grade-subject pairs using the OpenAI API.
- The function takes a list of grade-subject pairs, constructs a unique prompt for each, and requests 10 age-appropriate assignment prompts for each pair.
- The prompts are displayed in Markdown format within Jupyter Notebooks for easy viewing.

#### Parameters:
- **pairs**: A list of tuples, where each tuple consists of:
  - **grade_level**: A string representing the grade level (e.g., "11" for 11th grade).
  - **subject**: A string representing the academic subject for which prompts are being generated (e.g., "US History").

#### Process:
1. **Prompt Construction**: For each grade-subject pair, the function builds a specific request prompt that asks the OpenAI model to generate 10 unique, challenging, and engaging assignment prompts appropriate for that grade level and subject.
2. **API Call**: The function sends this constructed prompt to the OpenAI API (`gpt-3.5-turbo`), instructing it to generate relevant assignment ideas.
3. **Formatting and Cleaning**: After receiving the response:
   - Each prompt is stripped of any leading numbering or formatting inconsistencies using regular expressions to ensure a clean display.
   - Only the first 10 prompts are kept, even if more are returned.
4. **Display**: Each prompt is then labeled (e.g., "Prompt 1," "Prompt 2") and displayed in Markdown format for readability.

#### Output:
- **Returns**: A dictionary (`all_prompts`) where:
  - Each key is a tuple of the form `(grade_level, subject)`.
  - Each value is a list of 10 formatted assignment prompts for the corresponding grade-subject pair.



In [3]:
# Function to generate assignment prompts based on a list of grade-subject pairs
def generate_prompts(pairs):
    all_prompts = {}
    for grade_level, subject in pairs:
        prompt_text = f"""
        Role: You are a {grade_level}th-grade {subject} teacher. Please create 10 unique assignment prompts.

        Each prompt should:
        - Be age-appropriate for {grade_level}th grade
        - Challenge students' understanding, encouraging critical thinking and engagement with the subject matter
        - Cover diverse themes or topics within {subject}
        
        Ensure each prompt is clear, concise, and engaging.
        """
        
        # Generate prompts using OpenAI API
        completion = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "You are a creative and insightful prompt generator for teachers."},
                {"role": "user", "content": prompt_text}
            ]
        )
        
        # Extract prompts and clean up formatting
        prompts_text = completion.choices[0].message['content']
        prompts_list = [re.sub(r'^\d+\.\s*', '', prompt.strip()) for prompt in prompts_text.split('\n') if prompt.strip()]
        prompts_list = prompts_list[:10]  # Keep exactly 10 prompts
        all_prompts[(grade_level, subject)] = prompts_list
        
        # Display prompts in Markdown format
        display(Markdown(f"### Generated Prompts for {grade_level}th Grade {subject}"))
        for i, prompt in enumerate(prompts_list, start=1):
            display(Markdown(f"**Prompt {i}:** {prompt}"))
    
    return all_prompts


### Function: `generate_sample_essay`

#### Description:
- This function generates a sample essay for each grade-subject pair based on a specific prompt and a randomly assigned quality level. 
- It leverages a previously generated list of prompts, selects a specified prompt, and instructs OpenAI to generate a sample essay typical of a student at that grade level and subject.

#### Parameters:
- **all_prompts**: A dictionary where each key is a tuple `(grade_level, subject)` and each value is a list of 10 assignment prompts generated for that grade-subject pair.
- **pairs**: A list of grade-subject tuples, where each tuple consists of:
  - **grade_level**: A string indicating the grade level (e.g., "11" for 11th grade).
  - **subject**: A string representing the academic subject (e.g., "US History").
- **prompt_index**: An integer indicating which prompt from the list to use for essay generation (default is 0, which selects the first prompt in the list).

#### Process:
1. **Prompt Selection**: For each grade-subject pair in `pairs`, the function retrieves the specified prompt from `all_prompts` using `prompt_index`.
2. **Quality Level Assignment**: A quality level (e.g., "A", "B", "C", "D", or "F") is randomly selected for the essay using `random.choice(quality_levels)`.
3. **Essay Generation Prompt**: An essay prompt is constructed based on the selected prompt and assigned quality level, simulating a student essay typical of that grade and subject.
4. **API Call**: The function sends the essay prompt to the OpenAI API (`gpt-3.5-turbo`), requesting a sample response.
5. **Error Handling**: If prompts are missing or if `prompt_index` is out of range, an error message is printed. Any API errors are caught and logged, and the essay output for that pair is set to `None`.
6. **Display and Storage**: Each generated essay, along with its quality level, is displayed in Markdown format and stored in a dictionary for further use.

#### Output:
- **Returns**: A dictionary (`all_essays`) where:
  - Each key is a tuple `(grade_level, subject)`.
  - Each value is a dictionary with:
    - **quality**: The randomly assigned quality level for the essay.
    - **essay**: The generated essay text or `None` if an error occurred.

#### Example Usage:
```python
grade_subject_pairs = [("11", "US History"), ("10", "Global History")]
all_prompts = generate_prompts(grade_subject_pairs)
all_essays = generate_sample_essay(all_prompts, grade_subject_pairs, prompt_index=0)


In [4]:
def generate_sample_essay_with_explanation(prompt, grade_level, subject, quality):
    combined_prompt = f"""
    You are a {grade_level}th-grade student in {subject} with a {quality} average. Any assignment you write will reflect your grade average.
    With this in mind, please write an essay that reflects a {quality} level, where:

    - A (90-100): Excellent
    - B (80-89): Good
    - C (70-79): Basic
    - D (60-69): Minimal
    - F (59 and below): Very limited

    **Essay Topic:** {prompt}

    Write your essay directly, without repeating the topic instructions. Provide only the content of the essay as if you were submitting it for a grade.

    After the essay, include an explanation (4-5 sentences) describing why this essay reflects a {quality} quality level for a {grade_level}th grade and {subject} assignment. 
    Use "Explanation:" to begin the explanation.
    """
    
    try:
        completion = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "You are a skilled educational content generator."},
                {"role": "user", "content": combined_prompt}
            ]
        )
        
        # Extract response and separate essay and explanation
        response = completion.choices[0].message['content'].strip()
        essay, explanation = response.split("Explanation:", 1)
        return essay.strip(), explanation.strip()
        
    except Exception as e:
        print(f"Error generating essay and explanation for {grade_level}th Grade {subject}, Prompt: {prompt}:", e)
        return None, None


In [5]:
# Step 1: Define pairs of grade level and subject
pairs = [(11, "US History"), (10, "Global History")]



### Calling `generate_prompts`

- Defines `grade_subject_pairs`, a list of grade level and subject combinations for which assignment prompts are needed.
- Calls `generate_prompts` with this list, generating and displaying 10 unique assignment prompts for each grade-subject pair in a readable Markdown format.
- The output, `all_prompts`, is a dictionary that stores prompts by grade-subject pair for further use.




In [6]:
# Step 2: Generate prompts for each grade-subject pair
all_prompts = generate_prompts(pairs)


### Generated Prompts for 11th Grade US History

**Prompt 1:** In a persuasive essay, argue whether the 1920s should be known as the "Roaring Twenties" or the "Decade of Discontent". Use specific historical events and societal changes to support your argument.

**Prompt 2:** Create a timeline showcasing the key events leading up to the Civil War. Explain how these events contributed to the growing tensions between the North and the South.

**Prompt 3:** Imagine you are a journalist reporting on the aftermath of the Watergate scandal. Write a newspaper article detailing the impact of the scandal on American politics and society.

**Prompt 4:** Design a political cartoon representing the debate over Manifest Destiny in the 19th century. Include symbols and captions to convey the different perspectives on westward expansion.

...

**Prompt 6:** Assignment Prompt: "Investigate the role of religion in shaping societies. Choose two major world religions (e.g., Islam, Buddhism) and compare their beliefs, practices, and influence on government and society. How have these religions contributed to both unity and division among followers?"

**Prompt 7:** Assignment Prompt: "Analyze the causes and consequences of decolonization in the 20th century. Choose a specific region or country that gained independence from colonial rule and examine the challenges faced during the decolonization process. How did nationalism and post-colonial identity shape the newly independent nations?"

**Prompt 8:** Assignment Prompt: "Examine the impact of the Industrial Revolution on global economics and society. Discuss how industrialization transformed production, labor, and urbanization. What were the social and environmental consequences of this period of rapid industrial growth?"

**Prompt 9:** Assignment Prompt: "Investigate the role of technology in shaping warfare throughout history. Choose two significant military conflicts (e.g., World War I, Vietnam War) and analyze how advancements in technology affected the strategies and outcomes of these wars. How has warfare evolved as a result of technological innovation?"

**Prompt 10:** Assignment Prompt: "Reflect on the concept of globalization and its effects on cultural identity. How has increased interconnectedness through trade, communication, and migration influenced the preservation and adaptation of cultural traditions? Provide examples of how globalization has both homogenized and diversified cultures in the contemporary world."

In [7]:
# Define possible quality levels
quality_levels = ["A", "B", "C", "D", "F"]

# Step 3: Select a random prompt and quality for each pair
selected_qualities = [random.choice(quality_levels) for _ in pairs]       # Select random quality for each pair
selected_qualities

['F', 'C']

In [8]:
# Select a random prompt for each grade-subject pair
selected_prompts = [random.choice(all_prompts[pair]) for pair in pairs]

# Display the selected prompts as a formatted list
for i, (pair, prompt) in enumerate(zip(pairs, selected_prompts), start=1):
    grade_level, subject = pair
    print(f"{i}. {grade_level}th Grade {subject} Prompt:")
    print(f"   {prompt}\n")


1. 11th Grade US History Prompt:
   Compare and contrast the goals and strategies of the civil rights movements led by Martin Luther King Jr. and Malcolm X. Reflect on their impact on the fight for racial equality in America.

2. 10th Grade Global History Prompt:
   Assignment Prompt: "Imagine you are a historian studying ancient civilizations. Choose two societies (e.g., Mesopotamia, Egypt, Indus Valley) and compare their political structures, social hierarchies, and religious beliefs. What factors influenced the development of these differences?"



### Calling `generate_sample_essay`

- Defines `quality_levels`, a list of possible grades ("A" through "F") to randomly assign to each generated essay.
- Calls `generate_sample_essay` with `all_prompts`, `grade_subject_pairs`, and a specified `prompt_index` (0), generating a sample essay for each grade-subject pair.
- Each essay is displayed with its corresponding quality level and prompt. The output, `all_essays`, is a dictionary that stores the generated essays and their assigned quality levels for further use.







In [9]:
# Step 4: Generate sample essays and explanations using the selected prompts and qualities
all_essays = {}
for i, pair in enumerate(pairs):
    grade_level, subject = pair
    prompt = selected_prompts[i]
    quality = selected_qualities[i]
    
    # Generate essay and explanation in a single prompt
    essay, explanation = generate_sample_essay_with_explanation(prompt, grade_level, subject, quality)
    
    # Store results
    all_essays[pair] = {
        "quality": quality,
        "prompt": prompt,
        "essay": essay,
        "explanation": explanation
    }
    
    # Display results in Markdown format
    display(Markdown(f"### Sample Essay for {grade_level}th Grade {subject}"))
    display(Markdown(f"**Prompt:** {prompt}"))
    display(Markdown(f"**Quality:** {quality}"))
    display(Markdown(essay))
    display(Markdown(f"**Explanation:** {explanation}"))



### Sample Essay for 11th Grade US History

**Prompt:** Compare and contrast the goals and strategies of the civil rights movements led by Martin Luther King Jr. and Malcolm X. Reflect on their impact on the fight for racial equality in America.

**Quality:** F

Martin Luther King Jr. and Malcolm X were two leaders in the civil rights movement. They had different goals and strategies. MLK believed in nonviolent protests, like sit-ins and marches. He wanted racial equality through peace. Malcolm X, on the other hand, believed in self-defense and violent resistance. He thought that Black people needed to defend themselves against white oppression. These two leaders had a significant impact on the fight for racial equality in America, but their approaches were very different.

**Explanation:** This essay reflects a F level quality for an 11th-grade US History assignment because it provides a very limited discussion on the topic. The essay lacks specific details, analysis, and critical thinking. It does not delve into the complexities of the civil rights movements led by Martin Luther King Jr. and Malcolm X, nor does it offer any nuanced comparison of their goals and strategies. The content is basic and lacks depth, failing to demonstrate a thorough understanding of the subject matter expected at an 11th-grade level.

### Sample Essay for 10th Grade Global History

**Prompt:** Assignment Prompt: "Imagine you are a historian studying ancient civilizations. Choose two societies (e.g., Mesopotamia, Egypt, Indus Valley) and compare their political structures, social hierarchies, and religious beliefs. What factors influenced the development of these differences?"

**Quality:** C

Ancient Mesopotamia and Egypt were two distinct societies with unique political structures, social hierarchies, and religious beliefs. Mesopotamia was characterized by city-states ruled by kings who claimed divine authority. In contrast, Egypt was unified under a pharaoh who was considered a god-king. The social hierarchy in Mesopotamia was stratified, with the king and nobles at the top, followed by priests, artisans, and farmers. In Egypt, there was a similar hierarchy, but the pharaoh held the highest position with absolute power. 

Religiously, Mesopotamians worshipped a pantheon of gods and goddesses, with the belief in the afterlife playing a significant role in their religious practices. In contrast, Egyptians held a polytheistic belief system centered around the concept of Ma'at, emphasizing harmony and balance. The development of these differences can be attributed to geographical factors such as the Nile River in Egypt, which fostered a more stable and centralized society compared to the Tigris and Euphrates rivers in Mesopotamia. Additionally, cultural exchanges and interactions with neighboring societies influenced the religious beliefs and social structures of both civilizations.

**Explanation:** This essay demonstrates a basic understanding of the topic by providing a comparison of the political structures, social hierarchies, and religious beliefs of ancient Mesopotamia and Egypt. The content is clear and organized, covering the required aspects of the assignment. However, the essay lacks in-depth analysis and critical thinking, offering surface-level explanations without delving into the complexities of the factors influencing the development of these differences. To achieve a higher grade, the essay would need to incorporate more detailed examples and demonstrate a deeper understanding of the historical context.

## Rubric 

In [10]:
file_path = '/Users/samuelgartenstein/Desktop/qmss_llm_innovation_lab/Rubrics/General History Rubric.md'

# Read and print content as plain text
with open(file_path, 'r') as file:
    content = file.read()
print(content)


# High School History Course Rubric

| Criteria                               | Poor                                                                                                                     | Satisfactory                                                                                                                                            | Good                                                                                                                                                    | Excellent                                                                                                                                               | Total Possible Points |
|----------------------------------------|---------------------------------------------------------------------------------------------------------------------------|-------------------------------------------------------------------------------------------------------------------------------------------------

### Rubric Grading

In [11]:
# Function to grade an essay based on a rubric stored in a variable
def grade_essay_with_rubric(essay, rubric, grade_level, subject):
    # Prepare the grading prompt with the rubric and essay
    prompt = f"""
    You are a knowledgeable grader for a high school {subject} course. Your task is to evaluate an essay based on the following rubric:

    {rubric}

    Here is the essay to be graded:
    ---
    {essay}
    ---
    
    For each criterion in the rubric:
    1. Assign a score out of 25 points in the format "Score: X/25".
    2. Provide a 2-3 sentence explanation of the score, referencing specific aspects of the essay.

    After evaluating each criterion, calculate:
    - The total score out of 100.
    - The letter grade based on the following scale:
      - A: 90-100
      - B: 80-89
      - C: 70-79
      - D: 60-69
      - F: 59 and below

    Format the output as follows:
    1. **Criterion Name**
       - Score: X/25
       - Explanation: [Explanation here]

    After all criteria, provide:
    - **Total Score**: X/100
    - **Letter Grade**: [Letter grade here]
    - Explanation: Provide a 4-5 sentence summary explaining the main reasons for the overall grade based on the criteria.
    """

    # Call the OpenAI API to get the grading response
    try:
        completion = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "You are an experienced grader familiar with high school history rubrics."},
                {"role": "user", "content": prompt}
            ]
        )
        
        # Extract and separate the explanation from the rest of the grading response
        grading_response = completion.choices[0].message['content'].strip()
        
        # Splitting the grading response into main content and explanation
        main_content, explanation = grading_response.split("Explanation:", 1)
        
        return main_content.strip(), explanation.strip()
    
    except Exception as e:
        print(f"Error grading the essay: {e}")
        return None, None


In [12]:
# Example of using the function to display results
for i, pair in enumerate(pairs):
    grade_level, subject = pair
    essay = all_essays[pair]["essay"]  # Assuming all_essays contains essays for each pair
    rubric = "Your rubric content here"  # Replace with the actual rubric text

    # Get the grading breakdown and explanation
    grading_content, explanation = grade_essay_with_rubric(essay, rubric, grade_level, subject)
    
    # Display results in Markdown format
    display(Markdown(f"### Grading Results for {grade_level}th Grade {subject}"))
    display(Markdown(f"**Essay:**"))
    display(Markdown(essay))
    display(Markdown(f"**Grading Breakdown:**"))
    display(Markdown(grading_content))
    display(Markdown(f"**Explanation:** {explanation}"))


### Grading Results for 11th Grade US History

**Essay:**

Martin Luther King Jr. and Malcolm X were two leaders in the civil rights movement. They had different goals and strategies. MLK believed in nonviolent protests, like sit-ins and marches. He wanted racial equality through peace. Malcolm X, on the other hand, believed in self-defense and violent resistance. He thought that Black people needed to defend themselves against white oppression. These two leaders had a significant impact on the fight for racial equality in America, but their approaches were very different.

**Grading Breakdown:**

**Rubric for Grading Essay:**

1. **Content Understanding**
   - Score: 20/25
   -

...

**Essay:**

Ancient Mesopotamia and Egypt were two distinct societies with unique political structures, social hierarchies, and religious beliefs. Mesopotamia was characterized by city-states ruled by kings who claimed divine authority. In contrast, Egypt was unified under a pharaoh who was considered a god-king. The social hierarchy in Mesopotamia was stratified, with the king and nobles at the top, followed by priests, artisans, and farmers. In Egypt, there was a similar hierarchy, but the pharaoh held the highest position with absolute power. 

Religiously, Mesopotamians worshipped a pantheon of gods and goddesses, with the belief in the afterlife playing a significant role in their religious practices. In contrast, Egyptians held a polytheistic belief system centered around the concept of Ma'at, emphasizing harmony and balance. The development of these differences can be attributed to geographical factors such as the Nile River in Egypt, which fostered a more stable and centralized society compared to the Tigris and Euphrates rivers in Mesopotamia. Additionally, cultural exchanges and interactions with neighboring societies influenced the religious beliefs and social structures of both civilizations.

**Grading Breakdown:**

**Criterion 1: Understanding of Political Structures**
- Score: 22/25
-

**Explanation:** The essay effectively contrasts the political structures of Mesopotamia and Egypt, highlighting the roles of kings and pharaohs in each society. It accurately portrays Mesopotamia as having city-states ruled by kings with divine authority and Egypt as unified under a pharaoh considered a god-king.

**Criterion 2: Understanding of Social Hierarchies**
- Score: 23/25
- Explanation: The essay demonstrates a strong understanding of the social hierarchies in both Mesopotamia and Egypt, outlining the positions of various classes in society and emphasizing the unique position of the pharaoh in Egyptian society. The comparison between the two civilizations is clear and well-supported.

**Criterion 3: Understanding of Religious Beliefs**
- Score: 24/25
- Explanation: The essay provides a detailed analysis of the religious beliefs in Mesopotamia and Egypt, highlighting the differences in worship practices, belief systems, and influences on societal values. The reference to geographical factors and cultural exchanges to explain these differences is insightful.

**Criterion 4: Analysis and Explanation**
- Score: 21/25
- Explanation: While the essay effectively compares and contrasts Mesopotamia and Egypt in terms of political structures, social hierarchies, and religious beliefs, there could have been a deeper analysis of how these factors influenced each society's development. Additionally, more specific examples could have been included to strengthen the argument.

**Total Score**: 90/100
**Letter Grade**: A
**Explanation**: The essay demonstrates a strong understanding of the key differences between Ancient Mesopotamia and Egypt in terms of political structures, social hierarchies, and religious beliefs. The analysis is well-supported with relevant details and comparisons, leading to a comprehensive evaluation. With minor improvements in analysis and inclusion of specific examples, the essay could further enhance its argumentation.